## Add utils as a module

In [18]:
import sys

if '../utils' not in sys.path:
    sys.path.append('../utils')
if '../' not in sys.path:
    sys.path.append('../')

# create an __init__.py folder in the utils folder if you haven't already
from utils import data_loading

# Loading libraries

In [2]:
# import os
# import numpy as np
# from pathlib import Path
# from PIL import Image'
# from tqdm import tqdm
# import matplotlib
# from matplotlib import pyplot as plt
# from nilearn import datasets
# from nilearn import plotting
# import torch
# from torch.utils.data import DataLoader, Dataset
# from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
# from torchvision import transforms
# from sklearn.decomposition import IncrementalPCA
# from sklearn.linear_model import LinearRegression
# from scipy.stats import pearsonr as corr

# Loading Data

In [17]:
#loading data for subject 1, first 10 images
lh, rh, images  = data_loading.load(1, 0, 10)

print(lh.shape)
print(images[0].shape)

(10, 19004)
(3, 425, 425)
